In [1]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
PATH = r'C:\Users\joaov\OneDrive\Documentos\rsna-intracranial-hemorrhage-detection\random_selected_dataset'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (180, 180)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            seed=42,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 2000 files belonging to 2 classes.


In [5]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                            shuffle=True,
                                                            seed=42,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)


Found 200 files belonging to 2 classes.


In [6]:
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                            shuffle=True,
                                                            seed=42,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 600 files belonging to 2 classes.


In [7]:
def adjust_ds(ds):
    AUTOTUNE = tf.data.AUTOTUNE
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    adjusted_ds = ds.map(lambda x, y: (normalization_layer(x), y))
    
    return adjusted_ds
    
    
train_dataset = adjust_ds(train_dataset)
validation_dataset = adjust_ds(validation_dataset)
test_dataset = adjust_ds(test_dataset)

In [8]:
num_classes = 1

resnet_model = Sequential()
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180,180,3),
    pooling='avg',
    classes=num_classes,
    weights='imagenet'
)

for layer in pretrained_model.layers:
  layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(num_classes, activation='sigmoid'))

In [9]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 512)              1049088   
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 1)                513       
 pper)                                                           
                                                                 
Total params: 24,637,313
Trainable params: 1,049,601
Non-trainable params: 23,587,712
____________________________________

In [10]:
def compile_model(learning_rate):
    resnet_model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
      loss=tf.losses.BinaryCrossentropy(),
      metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    

In [11]:
def fit_model(train_dataset, validation_dataset, epochs):
    history = resnet_model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs)
    return history

In [12]:
learning_rates = [0.0001]

for index, learning_rate in enumerate(learning_rates):
    compile_model(learning_rate)
    history = fit_model(train_dataset, validation_dataset, 50)
    
    PATH_ = r'C:\Users\joaov\Downloads'
    NAME = f'resnet50_{index}_lr{learning_rate}.csv'
    df = pd.DataFrame(history.history)
    df.to_csv(os.path.join(PATH_, NAME))

Epoch 1/50
63/63 [==============================] - 100s 2s/step - loss: 0.6998 - accuracy: 0.5245 - precision: 0.5241 - recall: 0.5330 - val_loss: 0.7049 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
63/63 [==============================] - 97s 2s/step - loss: 0.6801 - accuracy: 0.5670 - precision: 0.5657 - recall: 0.5770 - val_loss: 0.6784 - val_accuracy: 0.5900 - val_precision: 0.5738 - val_recall: 0.7000
Epoch 3/50
63/63 [==============================] - 97s 2s/step - loss: 0.6701 - accuracy: 0.5885 - precision: 0.5833 - recall: 0.6200 - val_loss: 0.6889 - val_accuracy: 0.5500 - val_precision: 0.5284 - val_recall: 0.9300
Epoch 4/50
63/63 [==============================] - 98s 2s/step - loss: 0.6658 - accuracy: 0.5915 - precision: 0.5867 - recall: 0.6190 - val_loss: 0.6708 - val_accuracy: 0.5900 - val_precision: 0.5738 - val_recall: 0.7000
Epoch 5/50
63/63 [==============================] - 99s 2s/step - loss: 0.6713 - accuracy: 0.5680 - pre

Epoch 38/50
63/63 [==============================] - 100s 2s/step - loss: 0.6169 - accuracy: 0.6155 - precision: 0.6061 - recall: 0.6600 - val_loss: 0.6524 - val_accuracy: 0.6050 - val_precision: 0.5724 - val_recall: 0.8300
Epoch 39/50
63/63 [==============================] - 100s 2s/step - loss: 0.6175 - accuracy: 0.6365 - precision: 0.6188 - recall: 0.7110 - val_loss: 0.6627 - val_accuracy: 0.6000 - val_precision: 0.5641 - val_recall: 0.8800
Epoch 40/50
63/63 [==============================] - 100s 2s/step - loss: 0.6245 - accuracy: 0.6275 - precision: 0.6269 - recall: 0.6300 - val_loss: 0.6845 - val_accuracy: 0.5850 - val_precision: 0.5515 - val_recall: 0.9100
Epoch 41/50
63/63 [==============================] - 100s 2s/step - loss: 0.6157 - accuracy: 0.6375 - precision: 0.6065 - recall: 0.7830 - val_loss: 0.6696 - val_accuracy: 0.5650 - val_precision: 0.7097 - val_recall: 0.2200
Epoch 42/50
63/63 [==============================] - 100s 2s/step - loss: 0.6171 - accuracy: 0.6335 - pr

In [13]:
predictions = resnet_model.predict(test_dataset)

19/19 [==============================] - 27s 1s/step


In [14]:
predictions = np.round(predictions)

In [15]:
test_dataset = test_dataset.unbatch()

In [16]:
labels = list(test_dataset.map(lambda x, y: y))
labels = np.array(labels)

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
cm = confusion_matrix(labels, predictions)
cm

array([[100, 200],
       [ 80, 220]], dtype=int64)

In [19]:
sensitivity = cm[0][0]/(cm[0][0] + cm[0][1])
sensitivity

0.3333333333333333

In [20]:
specificity = cm[1][1]/(cm[1][1] + cm[1][0])
specificity

0.7333333333333333

In [21]:
accuracy = (cm[0][0] + cm[1][1])/cm.sum()
accuracy

0.5333333333333333